In [2]:
from dotenv import load_dotenv
import os
import requests
import base64

load_dotenv()

# Configuration
API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
IMAGE_PATH = "images/shelf.jpg"
encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
}

In [3]:
# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are a retail Coca-cola planogram and assortment auditor.  Our products SKUs are: Coca-cola Original can 330ml, Coca-cola Zero can 330ml, Fanta Orange can 330ml, Sprite Original can 330ml, Sprite Zero can 330ml, Dasani water bottle 500ml.  Your job is to exhaustively inspect each shelf of the cooler photo and audit to determine if the cooler is compliant or not by shelf and if there are products from competitors or different products."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "TASK:  you are going to create a single audit table by each shelf with columns: Shelf Number, Required SKUs, Compliance summary with counts per product by comparing the below RULES of the PLANOGRAM with the above attached image, the image has 6 shelfs.  DO NOT hallucinate with counts, include only products with numbers on the same shelf and same SKU similarity."
        },
        {
          "type": "text",
          "text": "\n"
        },
        {
          "type": "text",
          "text": "PLANOGRAM RULES:"
        },
        {
          "type": "text",
          "text": "Shelf 1 (top):"
        },
        {
          "type": "text",
          "text": "\t- Must have 2 units of Coca-Cola Original bottle 330ml"
        },
        {
          "type": "text",
          "text": "\t- Must have 2 units of Coca-Cola Zero bottle 330ml"
        },
        {
          "type": "text",
          "text": "Shelf 2:"
        },
        {
          "type": "text",
          "text": "\t- Must have 3 units of Fanta Orange bottle 330ml"
        },
        {
          "type": "text",
          "text": "Shelf 3:"
        },
        {
          "type": "text",
          "text": "\t- Must have 8 units of Coca-Cola original can 330ml"
        },
        {
          "type": "text",
          "text": "Shelf 4:"
        },
        {
          "type": "text",
          "text": "\t- Must have 8 units of Coca-Cola original can 330ml"
        },
        {
          "type": "text",
          "text": "Shelf 5:"
        },
        {
          "type": "text",
          "text": "\t- Must have 8 units of Coca-Cola original can 330ml"
        },
        {
          "type": "text",
          "text": "Shelf 6 (bottom)"
        },
        {
          "type": "text",
          "text": "\t- Must have 16 units of Dasani Water bottle 550ml"
        },
        {
          "type": "text",
          "text": "\n"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_image}"
          }
        },
        {
          "type": "text",
          "text": "\n"
        }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

In [4]:
ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')

# Send request
try:
    response = requests.post(ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
print(response.json())

{'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '| Shelf Number | Required SKUs                         | Compliance Summary                             |\n|--------------|--------------------------------------|------------------------------------------------|\n| 1            | 2 units of Coca-Cola Original bottle 330ml, 2 units of Coca-Cola Zero bottle 330ml | 2 Coca-Cola Original bottle 330ml, 2 Coca-Cola Zero bottle 330ml  |\n| 2            | 3 units of Fanta Orange bottle 330ml | 1 Fanta Orange bottle 330ml, 3 Sprite bottle 330ml, 2 Gatorade bottles, 1 Tropicana bottle  |\n| 3            | 8 units of Coca-Cola original can 330ml | 2 Coca-Cola original can 330ml, 1 Sprite original can 330ml, 1 Sprite Zero can

In [5]:
import json

# Print the response JSON nicely
print(json.dumps(response.json(), indent=4))

{
    "choices": [
        {
            "content_filter_results": {
                "hate": {
                    "filtered": false,
                    "severity": "safe"
                },
                "self_harm": {
                    "filtered": false,
                    "severity": "safe"
                },
                "sexual": {
                    "filtered": false,
                    "severity": "safe"
                },
                "violence": {
                    "filtered": false,
                    "severity": "safe"
                }
            },
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "| Shelf Number | Required SKUs                         | Compliance Summary                             |\n|--------------|--------------------------------------|------------------------------------------------|\n| 1            | 2 units of Coca-Cola Original bottle 330

In [7]:
# Extract and print the 'choices' -> 'message' -> 'content' part of the response JSON
choices_content = response.json().get('choices', [{}])[0].get('message', {}).get('content')
print(choices_content)

| Shelf Number | Required SKUs                         | Compliance Summary                             |
|--------------|--------------------------------------|------------------------------------------------|
| 1            | 2 units of Coca-Cola Original bottle 330ml, 2 units of Coca-Cola Zero bottle 330ml | 2 Coca-Cola Original bottle 330ml, 2 Coca-Cola Zero bottle 330ml  |
| 2            | 3 units of Fanta Orange bottle 330ml | 1 Fanta Orange bottle 330ml, 3 Sprite bottle 330ml, 2 Gatorade bottles, 1 Tropicana bottle  |
| 3            | 8 units of Coca-Cola original can 330ml | 2 Coca-Cola original can 330ml, 1 Sprite original can 330ml, 1 Sprite Zero can 330ml, 1 Fanta Orange can 330ml, 1 Canada Dry can 330ml, 1 A&W can 330ml, 1 other can 330ml  |
| 4            | 8 units of Coca-Cola original can 330ml | 1 Coca-Cola original can 330ml, 1 Sprite original can 330ml, 1 Fanta Orange can 330ml, 1 Canada Dry can 330ml, 3 other cans 330ml  |
| 5            | 8 units of Coca-Cola origin